<a href="https://colab.research.google.com/github/SaquibKhan-DS/311-Customer-Service-Optimization/blob/main/src/nyc_311_analyzer_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Professional Visualization Module for NYC 311 Service Analysis
Enhanced visualizations for business presentations and portfolio
"""

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

class NYC311Visualizer:
    """
    Professional visualization class for NYC 311 analysis.
    Creates publication-ready charts for business presentations.
    """

    def __init__(self, analyzer):
        """
        Initialize with NYC311Analyzer instance.

        Parameters:
        -----------
        analyzer : NYC311Analyzer
            Initialized analyzer with processed data
        """
        self.analyzer = analyzer
        self.df = analyzer.processed_df

        # Professional color schemes
        self.colors_primary = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
        self.colors_sequential = ['#08519c', '#3182bd', '#6baed6', '#9ecae1', '#c6dbef', '#eff3ff']
        self.colors_diverging = ['#d73027', '#fc8d59', '#fee08b', '#e0f3f8', '#91bfdb', '#4575b4']

        # Set professional styling
        plt.style.use('seaborn-v0_8-whitegrid')
        sns.set_palette(self.colors_primary)

    def create_executive_dashboard(self, save_path=None):
        """
        Create a professional executive dashboard with key metrics.

        Parameters:
        -----------
        save_path : str, optional
            Path to save the dashboard image

        Returns:
        --------
        plotly.graph_objects.Figure
        """
        # Get metrics and analysis data
        metrics = self.analyzer.calculate_service_metrics()
        temporal_data = self.analyzer.analyze_temporal_patterns()
        dept_data = self.analyzer.department_efficiency_analysis()

        # Create subplots
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Daily Request Volume & Response Time',
                'Top 10 Complaint Types',
                'Department Performance Comparison',
                'Peak Hours Analysis'
            ),
            specs=[
                [{"secondary_y": True}, {"type": "bar"}],
                [{"type": "scatter"}, {"secondary_y": True}]
            ]
        )

        # Chart 1: Daily patterns with dual y-axis
        daily_data = temporal_data['daily_patterns']
        fig.add_trace(
            go.Bar(
                x=daily_data.index,
                y=daily_data['request_count'],
                name='Request Volume',
                marker_color='#1f77b4',
                opacity=0.7
            ),
            row=1, col=1, secondary_y=False
        )

        fig.add_trace(
            go.Scatter(
                x=daily_data.index,
                y=daily_data['avg_response_hours'],
                name='Avg Response Time',
                line=dict(color='#ff7f0e', width=3),
                marker=dict(size=8)
            ),
            row=1, col=1, secondary_y=True
        )

        # Chart 2: Top complaint types
        top_complaints = self.df['Complaint Type'].value_counts().head(10)
        fig.add_trace(
            go.Bar(
                x=top_complaints.values,
                y=top_complaints.index,
                orientation='h',
                name='Complaint Volume',
                marker_color='#2ca02c'
            ),
            row=1, col=2
        )

        # Chart 3: Department efficiency scatter
        top_depts = dept_data.head(10)
        fig.add_trace(
            go.Scatter(
                x=top_depts['total_requests'],
                y=top_depts['avg_response_hours'],
                mode='markers+text',
                text=top_depts.index,
                textposition="top center",
                marker=dict(
                    size=top_depts['sla_24h_compliance'],
                    color=top_depts['efficiency_score'],
                    colorscale='RdYlGn_r',
                    showscale=True,
                    colorbar=dict(title="Efficiency Score")
                ),
                name='Departments'
            ),
            row=2, col=1
        )

        # Chart 4: Hourly patterns
        hourly_data = temporal_data['hourly_patterns']
        fig.add_trace(
            go.Bar(
                x=hourly_data.index,
                y=hourly_data['request_count'],
                name='Hourly Requests',
                marker_color='#9467bd',
                opacity=0.7
            ),
            row=2, col=2, secondary_y=False
        )

        fig.add_trace(
            go.Scatter(
                x=hourly_data.index,
                y=hourly_data['avg_response_hours'],
                name='Response Time',
                line=dict(color='#d62728', width=2),
                yaxis='y4'
            ),
            row=2, col=2, secondary_y=True
        )

        # Update layout
        fig.update_layout(
            height=800,
            title_text="NYC 311 Service Performance Dashboard",
            title_x=0.5,
            title_font_size=24,
            showlegend=False,
            template='plotly_white'
        )

        # Update axis labels
        fig.update_yaxes(title_text="Request Count", row=1, col=1, secondary_y=False)
        fig.update_yaxes(title_text="Response Time (Hours)", row=1, col=1, secondary_y=True)
        fig.update_xaxes(title_text="Request Volume", row=1, col=2)
        fig.update_xaxes(title_text="Total Requests", row=2, col=1)
        fig.update_yaxes(title_text="Avg Response Time (Hours)", row=2, col=1)
        fig.update_xaxes(title_text="Hour of Day", row=2, col=2)
        fig.update_yaxes(title_text="Request Count", row=2, col=2, secondary_y=False)
        fig.update_yaxes(title_text="Response Time (Hours)", row=2, col=2, secondary_y=True)

        if save_path:
            fig.write_html(save_path.replace('.png', '.html'))
            fig.write_image(save_path, width=1200, height=800, scale=2)

        return fig

    def plot_response_time_distribution(self, save_path=None):
        """
        Create professional response time distribution plot.

        Parameters:
        -----------
        save_path : str, optional
            Path to save the plot

        Returns:
        --------
        matplotlib.figure.Figure
        """
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('Response Time Distribution Analysis', fontsize=16, fontweight='bold')

        # Overall distribution
        sns.histplot(
            data=self.df,
            x='elapsed_time_hours',
            bins=50,
            ax=axes[0,0],
            color=self.colors_primary[0],
            alpha=0.7
        )
        axes[0,0].set_title('Overall Response Time Distribution')
        axes[0,0].set_xlabel('Response Time (Hours)')
        axes[0,0].axvline(self.df['elapsed_time_hours'].median(), color='red', linestyle='--', label=f'Median: {self.df["elapsed_time_hours"].median():.1f}h')
        axes[0,0].legend()

        # Box plot by complaint type (top 10)
        top_complaints = self.df['Complaint Type'].value_counts().head(10).index
        df_top = self.df[self.df['Complaint Type'].isin(top_complaints)]

        sns.boxplot(
            data=df_top,
            y='Complaint Type',
            x='elapsed_time_hours',
            ax=axes[0,1],
            palette='husl'
        )
        axes[0,1].set_title('Response Time by Complaint Type (Top 10)')
        axes[0,1].set_xlabel('Response Time (Hours)')

        # Time series of response times
        monthly_response = self.df.groupby(self.df['created_dt'].dt.to_period('M'))['elapsed_time_hours'].mean()
        monthly_response.plot(kind='line', ax=axes[1,0], color=self.colors_primary[2], linewidth=2, marker='o')
        axes[1,0].set_title('Monthly Average Response Time Trend')
        axes[1,0].set_xlabel('Month')
        axes[1,0].set_ylabel('Avg Response Time (Hours)')
        axes[1,0].tick_params(axis='x', rotation=45)

        # SLA compliance visualization
        sla_thresholds = [24, 48, 168]  # 1 day, 2 days, 1 week
        sla_compliance = []
        labels = ['24 Hours', '48 Hours', '1 Week']

        for threshold in sla_thresholds:
            compliance = (self.df['elapsed_time_hours'] <= threshold).mean() * 100
            sla_compliance.append(compliance)

        bars = axes[1,1].bar(labels, sla_compliance, color=self.colors_primary[:3], alpha=0.8)
        axes[1,1].set_title('SLA Compliance Rates')
        axes[1,1].set_ylabel('Compliance Rate (%)')
        axes[1,1].set_ylim(0, 100)

        # Add percentage labels on bars
        for bar, pct in zip(bars, sla_compliance):
            axes[1,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                          f'{pct:.1f}%', ha='center', va='bottom', fontweight='bold')

        plt.tight_layout()

        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')

        return fig

    def plot_geographic_analysis(self, save_path=None):
        """
        Create professional geographic analysis visualizations.

        Parameters:
        -----------
        save_path : str, optional
            Path to save the plot

        Returns:
        --------
        matplotlib.figure.Figure
        """
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        fig.suptitle('Geographic Distribution Analysis', fontsize=16, fontweight='bold')

        # City-wise request volume
        city_counts = self.df['City'].value_counts().head(15)
        city_counts.plot(kind='barh', ax=axes[0,0], color=self.colors_primary[0])
        axes[0,0].set_title('Request Volume by City (Top 15)')
        axes[0,0].set_xlabel('Number of Requests')

        # City-wise average response time
        city_response = self.df.groupby('City')['elapsed_time_hours'].mean().sort_values(ascending=False).head(15)
        city_response.plot(kind='barh', ax=axes[0,1], color=self.colors_primary[1])
        axes[0,1].set_title('Average Response Time by City (Top 15)')
        axes[0,1].set_xlabel('Average Response Time (Hours)')

        # Borough analysis if available
        if 'Borough' in self.df.columns:
            borough_metrics = self.df.groupby('Borough').agg({
                'Unique Key': 'count',
                'elapsed_time_hours': 'mean'
            }).sort_values('Unique Key', ascending=False)

            # Bubble plot: Volume vs Response Time by Borough
            scatter = axes[1,0].scatter(
                borough_metrics['Unique Key'],
                borough_metrics['elapsed_time_hours'],
                s=[x/1000 for x in borough_metrics['Unique Key']],  # Size based on volume
                alpha=0.6,
                c=range(len(borough_metrics)),
                cmap='viridis'
            )

            # Add borough labels
            for i, borough in enumerate(borough_metrics.index):
                axes[1,0].annotate(
                    borough,
                    (borough_metrics.iloc[i]['Unique Key'], borough_metrics.iloc[i]['elapsed_time_hours']),
                    xytext=(5, 5), textcoords='offset points', fontsize=10
                )

            axes[1,0].set_title('Borough Performance: Volume vs Response Time')
            axes[1,0].set_xlabel('Request Volume')
            axes[1,0].set_ylabel('Average Response Time (Hours)')

        # Geographic concentration analysis
        city_market_share = (self.df['City'].value_counts() / len(self.df) * 100).head(10)

        # Create pie chart for top cities
        colors_pie = plt.cm.Set3(np.linspace(0, 1, len(city_market_share)))
        wedges, texts, autotexts = axes[1,1].pie(
            city_market_share.values,
            labels=city_market_share.index,
            autopct='%1.1f%%',
            colors=colors_pie,
            startangle=90
        )

        axes[1,1].set_title('Market Share by City (Top 10)')

        # Make percentage text bold
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_fontweight('bold')

        plt.tight_layout()

        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')

        return fig

    def plot_department_performance(self, save_path=None):
        """
        Create comprehensive department performance analysis.

        Parameters:
        -----------
        save_path : str, optional
            Path to save the plot

        Returns:
        --------
        matplotlib.figure.Figure
        """
        dept_data = self.analyzer.department_efficiency_analysis()
        top_depts = dept_data.head(10)

        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        fig.suptitle('Department Performance Analysis', fontsize=16, fontweight='bold')

        # Volume vs Response Time Scatter
        scatter = axes[0,0].scatter(
            top_depts['total_requests'],
            top_depts['avg_response_hours'],
            s=top_depts['sla_24h_compliance'] * 5,  # Size based on SLA compliance
            c=top_depts['efficiency_score'],
            cmap='RdYlGn_r',
            alpha=0.7,
            edgecolors='black',
            linewidth=0.5
        )

        # Add department labels
        for i, dept in enumerate(top_depts.index):
            if len(dept) > 20:  # Truncate long department names
                dept_label = dept[:20] + '...'
            else:
                dept_label = dept
            axes[0,0].annotate(
                dept_label,
                (top_depts.iloc[i]['total_requests'], top_depts.iloc[i]['avg_response_hours']),
                xytext=(5, 5), textcoords='offset points', fontsize=8
            )

        axes[0,0].set_title('Department Efficiency: Volume vs Response Time')
        axes[0,0].set_xlabel('Total Requests')
        axes[0,0].set_ylabel('Average Response Time (Hours)')

        # Add colorbar
        cbar = plt.colorbar(scatter, ax=axes[0,0])
        cbar.set_label('Efficiency Score (Lower = Better)')

        # SLA Compliance by Department
        sla_data = top_depts['sla_24h_compliance'].sort_values(ascending=True)
        bars = axes[0,1].barh(range(len(sla_data)), sla_data.values, color=self.colors_primary[2])
        axes[0,1].set_yticks(range(len(sla_data)))
        axes[0,1].set_yticklabels([name[:25] + '...' if len(name) > 25 else name for name in sla_data.index])
        axes[0,1].set_title('24-Hour SLA Compliance by Department')
        axes[0,1].set_xlabel('SLA Compliance Rate (%)')

        # Add percentage labels
        for i, (bar, pct) in enumerate(zip(bars, sla_data.values)):
            axes[0,1].text(pct + 0.5, bar.get_y() + bar.get_height()/2,
                          f'{pct:.1f}%', ha='left', va='center', fontsize=9)

        # Market Share Analysis
        market_share = top_depts['market_share_pct'].sort_values(ascending=False)
        axes[1,0].bar(range(len(market_share)), market_share.values, color=self.colors_primary[3])
        axes[1,0].set_xticks(range(len(market_share)))
        axes[1,0].set_xticklabels([name[:15] + '...' if len(name) > 15 else name for name in market_share.index],
                                 rotation=45, ha='right')
        axes[1,0].set_title('Market Share by Department')
        axes[1,0].set_ylabel('Market Share (%)')

        # Efficiency Score Ranking
        efficiency_ranking = top_depts['efficiency_score'].sort_values(ascending=True)
        colors_eff = ['green' if score <= 1.5 else 'orange' if score <= 2.0 else 'red' for score in efficiency_ranking.values]

        bars_eff = axes[1,1].barh(range(len(efficiency_ranking)), efficiency_ranking.values, color=colors_eff)
        axes[1,1].set_yticks(range(len(efficiency_ranking)))
        axes[1,1].set_yticklabels([name[:25] + '...' if len(name) > 25 else name for name in efficiency_ranking.index])
        axes[1,1].set_title('Department Efficiency Scores (Lower = Better)')
        axes[1,1].set_xlabel('Efficiency Score')

        # Add efficiency score labels
        for i, (bar, score) in enumerate(zip(bars_eff, efficiency_ranking.values)):
            axes[1,1].text(score + 0.05, bar.get_y() + bar.get_height()/2,
                          f'{score:.2f}', ha='left', va='center', fontsize=9, fontweight='bold')

        plt.tight_layout()

        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')

        return fig

    def create_business_impact_visualization(self, save_path=None):
        """
        Create visualization showing potential business impact and cost savings.

        Parameters:
        -----------
        save_path : str, optional
            Path to save the plot

        Returns:
        --------
        matplotlib.figure.Figure
        """
        cost_analysis = self.analyzer.calculate_cost_impact()

        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('Business Impact & Cost Optimization Analysis', fontsize=16, fontweight='bold')

        # Cost savings scenarios
        scenarios = list(cost_analysis.keys())
        savings = [cost_analysis[scenario]['annual_cost_savings'] for scenario in scenarios]
        improvements = [cost_analysis[scenario]['improvement_percentage'] for scenario in scenarios]

        bars = axes[0,0].bar(range(len(scenarios)), [s/1000 for s in savings], color=self.colors_primary[:len(scenarios)])
        axes[0,0].set_xticks(range(len(scenarios)))
        axes[0,0].set_xticklabels(scenarios, rotation=45, ha='right')
        axes[0,0].set_title('Annual Cost Savings Potential')
        axes[0,0].set_ylabel('Cost Savings ($000s)')

        # Add value labels on bars
        for i, (bar, saving) in enumerate(zip(bars, savings)):
            axes[0,0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(savings)/1000 * 0.01,
                          f'${saving/1000:.0f}K', ha='center', va='bottom', fontweight='bold')

        # Response time improvement visualization
        current_time = self.df['elapsed_time_hours'].mean()
        improved_times = [current_time * 0.9, current_time * 0.75, self.df['elapsed_time_hours'].quantile(0.25)]
        scenario_labels = ['10% Improvement', '25% Improvement', 'Best Practice']

        x_pos = range(len(scenario_labels))
        axes[0,1].bar(x_pos, [current_time] * len(scenario_labels),
                     color='lightcoral', alpha=0.7, label='Current')
        axes[0,1].bar(x_pos, improved_times,
                     color='lightgreen', alpha=0.8, label='Improved')

        axes[0,1].set_xticks(x_pos)
        axes[0,1].set_xticklabels(scenario_labels)
        axes[0,1].set_title('Response Time Improvement Scenarios')
        axes[0,1].set_ylabel('Average Response Time (Hours)')
        axes[0,1].legend()

        # ROI Analysis
        implementation_costs = [50000, 100000, 200000]  # Estimated implementation costs
        roi_percentages = [(saving - cost)/cost * 100 for saving, cost in zip(savings[:3], implementation_costs)]

        colors_roi = ['green' if roi > 200 else 'orange' if roi > 100 else 'red' for roi in roi_percentages]
        bars_roi = axes[1,0].bar(range(len(scenario_labels)), roi_percentages, color=colors_roi)
        axes[1,0].set_xticks(range(len(scenario_labels)))
        axes[1,0].set_xticklabels(scenario_labels)
        axes[1,0].set_title('Return on Investment (ROI)')
        axes[1,0].set_ylabel('ROI (%)')
        axes[1,0].axhline(y=100, color='red', linestyle='--', alpha=0.7, label='Break-even')
        axes[1,0].legend()

        # Add ROI labels
        for bar, roi in zip(bars_roi, roi_percentages):
            axes[1,0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5,
                          f'{roi:.0f}%', ha='center', va='bottom', fontweight='bold')

        # Current vs Optimized Performance Metrics
        metrics = self.analyzer.calculate_service_metrics()
        current_metrics = [
            metrics['avg_response_time_hours'],
            metrics['sla_24h_compliance'],
            metrics['daily_avg_requests']
        ]

        optimized_metrics = [
            current_metrics[0] * 0.75,  # 25% improvement in response time
            min(current_metrics[1] * 1.3, 95),  # 30% improvement in SLA (capped at 95%)
            current_metrics[2] * 1.1  # 10% increase in capacity
        ]

        metric_names = ['Avg Response\nTime (hrs)', 'SLA Compliance\n(%)', 'Daily Capacity\n(requests)']
        x_pos = np.arange(len(metric_names))
        width = 0.35

        # Normalize metrics for comparison (different scales)
        current_norm = [current_metrics[0]/10, current_metrics[1], current_metrics[2]/100]
        optimized_norm = [optimized_metrics[0]/10, optimized_metrics[1], optimized_metrics[2]/100]

        axes[1,1].bar(x_pos - width/2, current_norm, width, label='Current', color='lightcoral', alpha=0.7)
        axes[1,1].bar(x_pos + width/2, optimized_norm, width, label='Optimized', color='lightgreen', alpha=0.8)

        axes[1,1].set_xticks(x_pos)
        axes[1,1].set_xticklabels(metric_names)
        axes[1,1].set_title('Current vs Optimized Performance')
        axes[1,1].set_ylabel('Normalized Performance Score')
        axes[1,1].legend()

        plt.tight_layout()

        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')

        return fig

    def create_interactive_timeline(self, save_path=None):
        """
        Create interactive timeline visualization using Plotly.

        Parameters:
        -----------
        save_path : str, optional
            Path to save the HTML file

        Returns:
        --------
        plotly.graph_objects.Figure
        """
        # Create monthly aggregations
        monthly_data = self.df.groupby(self.df['created_dt'].dt.to_period('M')).agg({
            'Unique Key': 'count',
            'elapsed_time_hours': 'mean',
            'Complaint Type': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'Unknown'
        }).reset_index()

        monthly_data['created_dt'] = monthly_data['created_dt'].dt.to_timestamp()

        # Create interactive plot
        fig = make_subplots(
            rows=2, cols=1,
            shared_xaxes=True,
            subplot_titles=('Monthly Request Volume', 'Average Response Time Trend'),
            vertical_spacing=0.1
        )

        # Request volume over time
        fig.add_trace(
            go.Scatter(
                x=monthly_data['created_dt'],
                y=monthly_data['Unique Key'],
                mode='lines+markers',
                name='Monthly Requests',
                line=dict(color='#1f77b4', width=3),
                marker=dict(size=8),
                hovertemplate='<b>%{x}</b><br>Requests: %{y:,}<extra></extra>'
            ),
            row=1, col=1
        )

        # Response time trend
        fig.add_trace(
            go.Scatter(
                x=monthly_data['created_dt'],
                y=monthly_data['elapsed_time_hours'],
                mode='lines+markers',
                name='Avg Response Time',
                line=dict(color='#ff7f0e', width=3),
                marker=dict(size=8),
                hovertemplate='<b>%{x}</b><br>Avg Response: %{y:.1f} hours<extra></extra>'
            ),
            row=2, col=1
        )

        # Update layout
        fig.update_layout(
            height=600,
            title_text="NYC 311 Service Trends Over Time",
            title_x=0.5,
            title_font_size=20,
            showlegend=True,
            template='plotly_white'
        )

        fig.update_xaxes(title_text="Date", row=2, col=1)
        fig.update_yaxes(title_text="Request Count", row=1, col=1)
        fig.update_yaxes(title_text="Response Time (Hours)", row=2, col=1)

        if save_path:
            fig.write_html(save_path)

        return fig

    def generate_all_visualizations(self, output_dir='visualizations/'):
        """
        Generate all professional visualizations and save them.

        Parameters:
        -----------
        output_dir : str, default 'visualizations/'
            Directory to save all visualizations

        Returns:
        --------
        dict: Dictionary of generated figure objects
        """
        import os
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        print("Generating professional visualizations...")

        figures = {}

        # Generate all visualizations
        print("1. Creating executive dashboard...")
        figures['dashboard'] = self.create_executive_dashboard(
            save_path=os.path.join(output_dir, 'executive_dashboard.html')
        )

        print("2. Creating response time analysis...")
        figures['response_time'] = self.plot_response_time_distribution(
            save_path=os.path.join(output_dir, 'response_time_analysis.png')
        )

        print("3. Creating geographic analysis...")
        figures['geographic'] = self.plot_geographic_analysis(
            save_path=os.path.join(output_dir, 'geographic_analysis.png')
        )

        print("4. Creating department performance...")
        figures['department'] = self.plot_department_performance(
            save_path=os.path.join(output_dir, 'department_performance.png')
        )

        print("5. Creating business impact visualization...")
        figures['business_impact'] = self.create_business_impact_visualization(
            save_path=os.path.join(output_dir, 'business_impact_analysis.png')
        )

        print("6. Creating interactive timeline...")
        figures['timeline'] = self.create_interactive_timeline(
            save_path=os.path.join(output_dir, 'interactive_timeline.html')
        )

        print(f"All visualizations saved to {output_dir}")

        return figures

# Usage Example
if __name__ == "__main__":
    print("Professional NYC 311 Visualization Module")
    print("=" * 45)
    print("Features:")
    print("- Executive dashboard with key metrics")
    print("- Response time distribution analysis")
    print("- Geographic pattern analysis")
    print("- Department performance comparison")
    print("- Business impact and cost analysis")
    print("- Interactive timeline visualizations")
    print("\nUse this with your NYC311Analyzer to create portfolio-ready visualizations!")